#Pre Requistics

## Mount Google Drive




In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My Drive/DeepProject
%ls
# !unzip training_dataset_homework1.zip

## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import sklearn.model_selection
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn import preprocessing

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

##Check If It Uses GPU

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Reproducibility

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Some Variables For Data Load

In [ ]:
dataset_dir = 'training_data_final'
labels = ['Species1', 
          'Species2',
          'Species3',
          'Species4',
          'Species5',
          'Species6',
          'Species7',
          'Species8',
]

## Callbacks and Some Hyperparameter 

In [ ]:
input_shape = (96, 96, 3)
epochs = 500
num_classes = 8
patience = 15
batch_size = 128

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# Load, Augment and preprocess the Data

In [ ]:
def create_dataset_PIL(img_folder):
    
    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= np.array(Image.open(image_path))
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array , class_name
PIL_img_data, class_name=create_dataset_PIL(dataset_dir)

In [ ]:
X,y = create_dataset_PIL(dataset_dir)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,stratify=y,test_size=0.15)

X_train = np.array(X_train)
X_test = np.array(X_test)

le = preprocessing.LabelEncoder()
le.fit(labels)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

y_train = tfk.utils.to_categorical(y_train)
y_test = tfk.utils.to_categorical(y_test)

In [ ]:
def preprocess(preprocessing_function = 'None', augment = True):
  #data augmentation
  if augment:
      width_shift_range = 0.3
      height_shift_range = 0.3
      horizontal_flip = True
      vertical_flip = True
  else:
      width_shift_range = 0.0
      height_shift_range = 0.0
      horizontal_flip = False
      vertical_flip = False


  if preprocessing_function == None:
    train_datagen = ImageDataGenerator(
                                      width_shift_range = width_shift_range,
                                      height_shift_range = height_shift_range,
                                      horizontal_flip = horizontal_flip,
                                      vertical_flip = vertical_flip,
                                      featurewise_center=True,
                                      featurewise_std_normalization=True,
                                      fill_mode = 'reflect',
                                        )
    val_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    train_datagen.fit(X_train)
    val_datagen.mean = train_datagen.mean
    val_datagen.std = train_datagen.std
  
  else:
      train_datagen = ImageDataGenerator(
                                      width_shift_range = width_shift_range,
                                      height_shift_range = height_shift_range,
                                      horizontal_flip = horizontal_flip,
                                      vertical_flip = vertical_flip,
                                      fill_mode = 'reflect',
                                      preprocessing_function = preprocessing_function)
      val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

                                      


  train_data = train_datagen.flow(X_train, y_train,batch_size = batch_size)
  val_data = val_datagen.flow(X_test, y_test)
  

  return train_data, val_data

# Transfer learning with EfficientnetB1 with three learning rates : [0.001,0.0002,0.00004]

In [ ]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
base_model1=tf.keras.applications.efficientnet.EfficientNetB1(include_top=False,weights='imagenet',input_shape=(96,96,3),classes=8)
print("Number of layers in the base model: ", len(base_model1.layers))

In [ ]:
aug_train_data, val_data = preprocess(augment = True,preprocessing_function= preprocess_input)

In [ ]:
base_model1.trainable = False
inputs = tfkl.Input(shape=input_shape)
x = base_model1(inputs)
x = tfkl.GlobalAveragePooling2D()(x)
x = tfkl.Dropout( 0.2,name='Dropout1', seed=seed)(x)
x = tfkl.Dense(512,activation='relu',kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.2, name='Dropout2',seed=seed)(x)
x= tfkl.Dense(units=512, name='Classifier2', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(x)
outputs = tfk.layers.Dense(8,activation='softmax',kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(x)
model1 = tfk.Model(inputs, outputs)

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics='accuracy')
model1.summary()

In [ ]:
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True )]
).history

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0002), metrics='accuracy')
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True )]
).history

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00004), metrics='accuracy')
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True )]
).history


# fine tuning EfficientnetB1 with three learning rates : [0.0005,0.00008,0.00002]

In [ ]:
base_model1.trainable = True

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0005), metrics='accuracy')
model1.summary()

In [ ]:
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00008), metrics='accuracy')
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

In [ ]:
model1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00002), metrics='accuracy')
history = model1.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

In [ ]:
model1.save('MyDrive/efficientnetb1_transferall')

# Transfer learning with EfficientnetB2 with three learning rates : [0.001,0.0002,0.00004]

In [ ]:
base_model2=tf.keras.applications.efficientnet.EfficientNetB2(include_top=False,weights='imagenet',input_shape=(96,96,3),classes=8)

In [ ]:
from tensorflow.python.framework.test_util import for_all_test_methods
base_model2.trainable = for_all_test_methods

inputs = tfkl.Input(shape=input_shape)
x = base_model2(inputs)
x = tfkl.GlobalAveragePooling2D()(x)
x = tfkl.Dropout( 0.2,name='Dropout1', seed=seed)(x)
x = tfkl.Dense(512,activation='relu',kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.2, name='Dropout2',seed=seed)(x)
x= tfkl.Dense(units=512, name='Classifier2', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(x)
outputs = tfk.layers.Dense(8,activation='softmax',kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(x)
model2 = tfk.Model(inputs, outputs)

In [ ]:
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics='accuracy')
model2.summary()

In [ ]:
history2 = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True )]
).history

In [ ]:
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0002), metrics='accuracy')
history = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

In [ ]:
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00004), metrics='accuracy')
history = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

# Fine Tuning the EffecientnetB2 with three learning rates: [0.0005,0.00008,0.00002]

In [ ]:
base_model2.trainable = True
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0005), metrics='accuracy')
model2.summary()

In [ ]:
history = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history


In [ ]:
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00008), metrics='accuracy')
history = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history


In [ ]:
model2.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.00002), metrics='accuracy')
history = model2.fit(
    x = aug_train_data,
    epochs = epochs,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history


In [ ]:
model2.save('MyDrive/efficientnetb2_transferall')

# Ensemble Model

In [ ]:
# load the two fine tuned models
efficientnetb2tl=tfk.models.load_model('efficientnetb2_transferall')
efficientnetb2tl._name='efficientnetb2tl'
efficientnetb1tl=tfk.models.load_model('efficientnetb1_transferall')
efficientnetb1tl._name='efficientnetb1tl'

In [ ]:
input=tfkl.Input(shape=input_shape,name='inputlayer_resnet')
x1=efficientnetb2tl(input)
x2=efficientnetb1tl(input)

x=tf.keras.layers.Concatenate()([x1,x2])
output_layer=tfkl.Dense(units=8,activation='softmax',name='output_layer2',kernel_initializer=tfk.initializers.HeUniform(seed))(x)
ensemble_model=tfk.Model(inputs=input,outputs=output_layer,name='model')
ensemble_model.get_layer('efficientnetb2tl').trainable=False
ensemble_model.get_layer('efficientnetb1tl').trainable=False

In [ ]:
ensemble_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [ ]:
history = ensemble_model.fit(
    x = aug_train_data,
    epochs = 200,
    validation_data = val_data,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True )]
).history

In [ ]:
ensemble_model.save('MyDrive/ensemble_efficientnet12')